# Making chloropleth maps in Altair

Here's a quick example of how to make a chloropleth map in Altair.  In this example, we'll work with a fairly large data set of baby names in France from 1900-2019, broken down by department.

To work with geographical data, we'll use the `geopandas`, which loads `pandas` dataframes, but with support for geographical outlines in the `geojson` format.  You can use these dataframes just as you would a regular `pandas` dataframe, but they will include that extra geographical outline data.

To get started, we'll need to import our libraries.

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

pass

# Reading our names data

Now, let's read in our dataset.  The exported data is in CSV format, but with a `;` separator instead of commas.  The INSEE data collapses rare names or where department-level information has been elided (presumably to protect individuals with uncommon names or who were one of the only ones born with that name in a given year).  We'll strip those out.

In [2]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
1619591,1,UGO,1992,34,5
1057663,1,LUDOVIC,1978,87,45
244395,1,BORIS,1997,94,3
3278426,2,NADIA,1950,36,5
3599393,2,SULPICE,1911,971,4


# Loading map data

Next, let's load some map data of regions in France using `geopandas`.  These map data come from the [INSEE] and [IGN] and were processed into the `geojson` format we'll need to work with by [Grégoire David].  Here's the [github] repository.

In this example, we'll work with the simplified departments tiles for the Hexagon, but that repository contains higher-resolution versions, the DOM-TOM, and more.

[Grégoire David]: https://gregoiredavid.fr
[INSEE]: http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement.asp
[IGN]: https://geoservices.ign.fr/adminexpress
[github]: https://github.com/gregoiredavid/france-geojson/

In [4]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
10,11,Aude,"POLYGON ((1.68842 43.27355, 1.70839 43.29127, ..."
70,70,Haute-Saône,"POLYGON ((5.88473 47.92605, 5.90011 47.94475, ..."
47,47,Lot-et-Garonne,"POLYGON ((0.29733 44.76228, 0.32329 44.74762, ..."
12,13,Bouches-du-Rhône,"POLYGON ((4.73906 43.92406, 4.82174 43.91283, ..."
18,19,Corrèze,"POLYGON ((1.89873 45.69828, 1.91552 45.71126, ..."


Notice how `depts` is a geopandas dataframe.  We'll use it just as a regular `pandas` dataframe, but it includes the geometry info we need to be able to draw those regions when we pass them into Altair.  We just need to make sure that when we work with our data, we keep them in a geopandas dataframe and not a plain dataframe if we want to draw the departments.

In the next cell, notice how we do a right-merge to bring in department data into names.  We do this as a merge on `depts` because we need a geopandas dataframe.  Remember, `depts` is a geopandas dataframe, while `names` is a regular dataframe.  If we did a left merge on `names`, we'd end up with a regular pandas dataframe. After this merge, both `names` and `depts` will be geopandas dataframes.

**Hint:** Be careful when you do your data joins here.  It's easy to accidentally merge the wrong way to accidentally create a _much bigger_ dataset.

In [5]:
# Keep a reference around to the plain pandas dataframe, without geometry data, just in case
just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')

names.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
2558011,40,Landes,"POLYGON ((-0.24284 43.58498, -0.2764 43.58476,...",2,INAYA,2009,40,5
1630845,27,Eure,"POLYGON ((0.29722 49.42986, 0.33898 49.44093, ...",1,WILLIAM,1961,27,20
1038576,50,Manche,"POLYGON ((-1.11962 49.35557, -1.11346 49.32795...",1,LUDOVIC,1994,50,5
2949994,33,Gironde,"POLYGON ((-0.7188 45.32742, -0.64431 45.32205,...",2,MAITE,1961,33,5
1396727,14,Calvados,"POLYGON ((-1.11962 49.35557, -1.07822 49.38849...",1,RENÉ,1950,14,33


# Show a name over all years

Now we'll choose a name to show across all years.  To that, we'll group all of the names in a department together (squashing the years together) and use the sum.

In [6]:
grouped = names.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum(numeric_only=True)
grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt') # Add geometry data back in
grouped

,code,nom,geometry,dpt,preusuel,sexe,nombre
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,AARON,1,160
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABBY,2,3
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDALLAH,1,7
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDEL,1,3
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDELKADER,1,3
...,...,...,...,...,...,...,...
239574,NaN,NaN,None,974,ÉSAÏE,1,3
239575,NaN,NaN,None,974,ÉTHAN,1,53
239576,NaN,NaN,None,974,ÉTIENNE,1,3
239577,NaN,NaN,None,974,ÉVA,2,32


Now let's pick a name and check out how it's distribution over the last 120 years across Metropolitan France.  In this example, I choose the name “Lucien,” which I rather like for some reason.

In [7]:
multi = alt.selection_multi()

name = 'LUCIEN'
subset = grouped[grouped.preusuel == name]
alt.Chart(subset).mark_geoshape(stroke='white').encode(
    tooltip=['nom', 'code', 'nombre'],
    color='nombre',
).properties(width=800, height=600).add_selection(
    multi
)

C:\Users\mlarn\AppData\Local\Temp\ipykernel_1884\2301346975.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  multi = alt.selection_multi()
C:\Users\mlarn\AppData\Local\Temp\ipykernel_1884\2301346975.py:8: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).properties(width=800, height=600).add_selection(


alt.Chart(...)

In [8]:
import pandas as pd
import geopandas as gpd
import altair as alt

In [10]:
# Visualization 2

names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)
depts = gpd.read_file('departements-version-simplifiee.geojson')


names = depts.merge(names, how='right', left_on='code', right_on='dpt')

df_grouped = names.groupby(['nom', 'preusuel'])['nombre'].sum().reset_index()

top_names_list = (
    df_grouped
    .groupby('nom')
    .apply(lambda x: x.nlargest(3, 'nombre')['preusuel'].tolist())
    .reset_index()
    .rename(columns={0: 'top_3_names'})
)

top_names_nombre_list = (
    df_grouped
    .groupby('nom')
    .apply(lambda x: x.nlargest(3, 'nombre')['nombre'].tolist())
    .reset_index()
    .rename(columns={0: 'nombre_list'})
)

top_names = top_names_list.merge(top_names_nombre_list, how='right', left_on='nom', right_on='nom')

gdf = depts.merge(top_names, how='right', left_on='nom', right_on='nom')

map = alt.Chart(gdf).mark_geoshape(stroke='white').properties(
    width=600,
    height=400
).encode(
        color=alt.value('lightgray'),
        tooltip=['nom', 'top_3_names', 'nombre_list'],
).properties(width=600, height=400, title='Popular names per region')

map

C:\Users\mlarn\AppData\Local\Temp\ipykernel_1884\4143814105.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(3, 'nombre')['preusuel'].tolist())
C:\Users\mlarn\AppData\Local\Temp\ipykernel_1884\4143814105.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(3, 'nombre')['nombre'].tolist())


alt.Chart(...)